# Combining Instrument Properties with Holding Properties using a Derived Property 

In this notebook we will show how you can combine property data from the instrument master with holding properties.
We will first add instruments and their properties to our instrument master, intentionally leaving some property values blank. Then we will be creating a portfolio with sub holding keys, where the sub holding keys will serve as holding properties. Finally, we will create a new derived property that will fill in the gaps of our instrument master's collection of properties with the sub holding key properties.

For more background on sub-holding keys and derived properties you can refer to the below knowledge base articles:

Sub-holding keys: https://support.lusid.com/knowledgebase/article/KA-01879/en-us

Derived properties: https://support.lusid.com/knowledgebase/article/KA-02192/en-us


In [129]:
# Import generic non-LUSID packages
import os
import pandas as pd
import numpy as np
from datetime import datetime
import json
import pytz
import time
from IPython.core.display import HTML

# Import key modules from the LUSID package
import lusid as lu
import lusid.models as lm

# Import key functions from Lusid-Python-Tools and other packages
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon.transaction_type_upload import upsert_transaction_type_alias
from lusidtools.cocoon.cocoon import load_from_data_frame
from lusidtools.lpt.lpt import to_date
from lusidjam import RefreshingToken
from lusidtools.cocoon.cocoon_printer import (
    format_instruments_response,
    format_portfolios_response,
    format_transactions_response,
    format_quotes_response,
    format_holdings_response
)

# Set DataFrame display formats
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:,.2f}".format
# display(HTML("<style>.container { width:90% !important; }</style>"))

# Set the secrets path
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Authenticate our user and create our API client
api_factory = lu.utilities.ApiClientFactory(
    token=RefreshingToken(), api_secrets_filename=secrets_path
)

print("LUSID Environment Initialised")
print(
    "LUSID API Version :",
    api_factory.build(lu.api.ApplicationMetadataApi).get_lusid_versions().build_version,
)

LUSID Environment Initialised
LUSID API Version : 0.6.8634.0


In [130]:
# LUSID Variable Definitions
portfolio_api = api_factory.build(lu.api.PortfoliosApi)
transaction_portfolios_api = api_factory.build(lu.api.TransactionPortfoliosApi)
instruments_api = api_factory.build(lu.api.InstrumentsApi)

In [131]:
scope = "ibor"

## Adding Instruments with their Properties to the Instrument Master

In [132]:
instrument_master = pd.read_csv('data/coalesce-demo-instrument-master.csv')
instrument_master

,Isin,Ticker,Name,Strategy,Sector
0,US5949181045,MSFT US,Microsoft,Value,NaN
1,US9311421039,WMT US,Walmart,NaN,Consumer Services
2,US4370761029,HD US,Home Depot,NaN,Consumer Services
3,US0378331005,AAPL US,Apple,Growth,NaN
4,US3453708600,F US,Ford Motor Company,Growth,Consumer Cyclical


In [133]:
instrument_identifier_mapping = {
    "ClientInternal": "Ticker",
    "Isin": "Isin",
}

instrument_mapping_required = {"name": "Name"}

responses = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=instrument_master,
    mapping_required=instrument_mapping_required,
    mapping_optional={},
    file_type="instrument",
    identifier_mapping=instrument_identifier_mapping,
    property_columns=[
        "Strategy",
        "Sector",
    ],
)

succ, failed, errors = format_instruments_response(responses)
pd.DataFrame(
    data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}]
)

,success,failed,errors
0,5,0,0


## Creating the Sub Holding Key Properties

In [134]:
domain = "Transaction"
scope = scope
prop_code = "Strategy"

try:
    api_factory.build(lu.api.PropertyDefinitionsApi).create_property_definition(
        create_property_definition_request=lm.CreatePropertyDefinitionRequest(
            domain=domain,
            scope=scope,
            code=prop_code,
            value_required=None,
            display_name="Investment strategy",
            data_type_id=lu.ResourceId(scope="system", code="string"),
            life_time=None,
        )
    )

except lu.exceptions.ApiException as e:
    print(json.loads(e.body)["title"])

In [135]:
domain = "Transaction"
scope = scope
prop_code = "Sector"

try:
    api_factory.build(lu.api.PropertyDefinitionsApi).create_property_definition(
        create_property_definition_request=lm.CreatePropertyDefinitionRequest(
            domain=domain,
            scope=scope,
            code=prop_code,
            value_required=None,
            display_name="Sector",
            data_type_id=lu.ResourceId(scope="system", code="string"),
            life_time=None,
        )
    )

except lu.exceptions.ApiException as e:
    print(json.loads(e.body)["title"])

## Creating the Portfolio

In [136]:
portfolio_df = pd.read_csv('data/coalesce-demo-portfolio.csv')
portfolio_df

,Code,Name,Currency,Created
0,coalescePortfolio,Coalesce Example Portfolio,GBP,2019-01-01T00:00:00+00:00


In [137]:
portfolio_mapping = {
    "required": {"code": "Code", "display_name": "Name", "base_currency": "Currency",},
    "optional": {"created": "Created"},
}

result = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=portfolio_df,
    mapping_required=portfolio_mapping["required"],
    mapping_optional=portfolio_mapping["optional"],
    file_type="portfolios",
    sub_holding_keys=[
        "Transaction/ibor/Strategy",
        "Transaction/ibor/Sector",
    ],
)

succ, failed = format_portfolios_response(result)
display(pd.DataFrame(data=[{"success": len(succ), "failed": len(failed)}]))

,success,failed
0,1,0


## Adding Transactions to the Portfolio

In [138]:
transactions = pd.read_csv("data/coalesce-demo-transactions.csv")
transactions

,txn_id,Strategy,Sector,type,Isin,Ticker,trade_date,settlement_date,quantity,price,total_consideration,currency,portfolio
0,txn001,Growth,Technology,Buy,US5949181045,MSFT US,2021-09-01T10:00:00Z,2021-09-02T10:00:00Z,"1,000.00",300.00,"300,000.00",USD,coalescePortfolio
1,txn002,Value,Consumer Services,Buy,US9311421039,WMT US,2021-09-01T10:00:00Z,2021-09-02T10:00:00Z,"2,705.00",148.00,"400,340.00",USD,coalescePortfolio
2,txn003,Value,Consumer Services,Buy,US4370761029,HD US,2021-09-01T10:00:00Z,2021-09-02T10:00:00Z,765,328.00,"250,920.00",USD,coalescePortfolio
3,txn004,Growth,Technology,Buy,US0378331005,AAPL US,2021-09-01T10:00:00Z,2021-09-02T10:00:00Z,"4,610.00",152.00,"700,720.00",USD,coalescePortfolio
4,txn005,Growth,Consumer Cyclical,Buy,US3453708600,F US,2021-09-01T10:00:00Z,2021-09-02T10:00:00Z,"9,400.00",13.30,"125,020.00",USD,coalescePortfolio


In [139]:
transaction_field_mapping_required = {
    "code": "portfolio",
    "transaction_id": "txn_id",
    "type": "type",
    "transaction_date": "trade_date",
    "settlement_date": "settlement_date",
    "units": "quantity",
    "transaction_price.price": "price",
    "transaction_price.type": "$Price",
    "total_consideration.amount": "total_consideration",
    "total_consideration.currency": "currency",
    "exchange_rate": "$1",
    "transaction_currency": "currency",
}


transaction_identifier_mapping = {
    "ClientInternal": "Ticker",
}

In [140]:
responses = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=transactions,
    mapping_required=transaction_field_mapping_required,
    mapping_optional={},
    identifier_mapping=transaction_identifier_mapping,
    file_type="transaction",
    property_columns = [
        "Strategy", 
        "Sector"
    ],
)

succ, failed = format_transactions_response(responses)
display(pd.DataFrame(data=[{"success": len(succ), "failed": len(failed)}]))

,success,failed
0,1,0


## Creating Derived Properties

In [141]:
try:
    api_factory.build(
        lu.api.PropertyDefinitionsApi
    ).create_derived_property_definition(
        create_derived_property_definition_request=lu.models.CreateDerivedPropertyDefinitionRequest(
            domain="Holding",
            scope=scope,
            code="DerivedStrategy",
            display_name="Strategy from SHK or Instrument properties",
            data_type_id=lu.ResourceId(scope="system", code="string"),
            derivation_formula=f"Coalesce(Properties[Instrument/{scope}/Strategy], SubHoldingKeys[Transaction/{scope}/Strategy], 'None')",
        )
    )
except lu.exceptions.ApiException as e:
    print(json.loads(e.body)["title"])

In [142]:
try:
    api_factory.build(
        lu.api.PropertyDefinitionsApi
    ).create_derived_property_definition(
        create_derived_property_definition_request=lu.models.CreateDerivedPropertyDefinitionRequest(
            domain="Holding",
            scope=scope,
            code="DerivedSector",
            display_name="Sector from SHK or Instrument properties",
            data_type_id=lu.ResourceId(scope="system", code="string"),
            derivation_formula=f"if(HoldingType eq 'C') then 'Cash Commitment' else Coalesce(Properties[Instrument/{scope}/Sector], SubHoldingKeys[Transaction/{scope}/Sector], 'None')",
        )
    )
except lu.exceptions.ApiException as e:
    print(json.loads(e.body)["title"])

## Retrieve the Portfolio Data

In [143]:
portfolio = transaction_portfolios_api.get_holdings(
        scope=scope,
        code="coalescePortfolio",
        property_keys=[
            f"Instrument/{scope}/Strategy", 
            f"Instrument/{scope}/Sector", 
            f"Holding/{scope}/DerivedStrategy", 
            f"Holding/{scope}/DerivedSector",
            "Instrument/default/ClientInternal",
            ]
    )

In [144]:
results = lusid_response_to_data_frame(portfolio)

# We filtered out the relevant columns to make our use case more clear and readable.
results[[
    'instrument_uid',
    'properties.Instrument/default/ClientInternal.value.label_value',
    'properties.Instrument/ibor/Sector.value.label_value',
    'sub_holding_keys.Transaction/ibor/Sector.value.label_value',
    'properties.Holding/ibor/DerivedSector.value.label_value',
    'properties.Instrument/ibor/Strategy.value.label_value',
    'sub_holding_keys.Transaction/ibor/Strategy.value.label_value',
    'properties.Holding/ibor/DerivedStrategy.value.label_value',   
    ]]

,instrument_uid,properties.Instrument/default/ClientInternal.value.label_value,properties.Instrument/ibor/Sector.value.label_value,sub_holding_keys.Transaction/ibor/Sector.value.label_value,properties.Holding/ibor/DerivedSector.value.label_value,properties.Instrument/ibor/Strategy.value.label_value,sub_holding_keys.Transaction/ibor/Strategy.value.label_value,properties.Holding/ibor/DerivedStrategy.value.label_value
0,LUID_00003D6E,MSFT US,NaN,Technology,Technology,Value,Growth,Value
1,LUID_00003D6D,WMT US,Consumer Services,Consumer Services,Consumer Services,NaN,Value,Value
2,LUID_00003D6H,HD US,Consumer Services,Consumer Services,Consumer Services,NaN,Value,Value
3,LUID_00003D6G,AAPL US,NaN,Technology,Technology,Growth,Growth,Growth
4,LUID_00003D6F,F US,Consumer Cyclical,Consumer Cyclical,Consumer Cyclical,Growth,Growth,Growth
5,CCY_USD,NaN,NaN,Technology,Technology,NaN,Growth,Growth
6,CCY_USD,NaN,NaN,Consumer Services,Consumer Services,NaN,Value,Value
7,CCY_USD,NaN,NaN,Consumer Cyclical,Consumer Cyclical,NaN,Growth,Growth


Here we can see the Instrument properties, which header starts with "properties.Instrument" and the Sub Holding Keys, which header starts with "sub_holding_keys.Transaction".
When we look at MSFT, we can see the NaN value for Sector in the instrument master ('properties.Instrument/ibor/Sector.value.label_value').
However, there is a value for Sector in the Sub Holding Keys ('sub_holding_keys.Transaction/ibor/Sector.value.label_value') and therefore we see the value "Technology" in the derived property for sector ('properties.Holding/ibor/DerivedSector.value.label_value').